In [3]:
# 이 코드에서는 얼굴 감정 분석을 위한 VGGNet 모델을 사용합니다.
# mediapipe를 이용해 이미지에서 얼굴을 감지하고 크롭한 후 VGGNet 모델을 활용하여 감정을 예측합니다.

import numpy as np
import cv2
import mediapipe as mp
import os
import glob

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [4]:
# 이미지 크롭 사이즈
CROP_SIZE = (224,224)
# 감정 카테고리 및 색상 설정
emotions = {
    '0': ['angry', (0,0,255), (255,255,255)],
    '1': ['embarrassed', (0,255,0), (255,255,255)],   # 초록색, 흰색 텍스트
    '2': ['happy', (255,0,0), (255,255,255)],  # 파란색, 흰색 텍스트
    '3': ['sad', (0,255,255), (0,0,0)],    # 노란색, 검은색 텍스트
    '4': ['neutral', (255,0,255), (0,0,0)]         # 보라색, 검은색 텍스트
}
num_classes = len(emotions)
input_shape = CROP_SIZE+(1,)

In [5]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes, checkpoint_path, lr=1e-3):
        super().__init__()
        self.add(Rescaling(1./255, input_shape=input_shape))
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
        self.add(BatchNormalization())
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Flatten())
        
        self.add(Dense(1024, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(256, activation='relu'))

        self.add(Dense(num_classes, activation='softmax'))

        self.compile(optimizer=Adam(learning_rate=lr),
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])
        
        self.checkpoint_path = checkpoint_path

In [6]:
weights = os.path.join(os.getcwd(),'models','vggnet_korean_trained.h5')
model = VGGNet(input_shape, num_classes, weights)
model.load_weights(model.checkpoint_path)

In [7]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [8]:
def detection_preprocessing(image, h_max=360):
    h, w, _ = image.shape
    if h > h_max:
        ratio = h_max / h
        w_ = int(w * ratio)
        image = cv2.resize(image, (w_,h_max))
    return image

def resize_face(face):
    x = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
    return tf.image.resize(x, (224,224))

def recognition_preprocessing(faces):
    x = tf.convert_to_tensor([resize_face(f) for f in faces])
    return x

In [9]:
def inference(image):
    H, W, _ = image.shape
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_image)

    if results.detections:
        faces = []
        pos = []
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            # mp_drawing.draw_detection(image, detection)

            x = int(box.xmin * W)
            y = int(box.ymin * H)
            w = int(box.width * W)
            h = int(box.height * H)

            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(x + w, W)
            y2 = min(y + h, H)

            face = image[y1:y2,x1:x2]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            faces.append(face)
            pos.append((x1, y1, x2, y2))
    
        x = recognition_preprocessing(faces)

        y = model.predict(x)
        l = str(np.argmax(y, axis=1)[0])

        for i in range(len(faces)):
            cv2.rectangle(image, (pos[i][0],pos[i][1]),
                            (pos[i][2],pos[i][3]), emotions[l[i]][1], 2, lineType=cv2.LINE_AA)
            
            cv2.rectangle(image, (pos[i][0],pos[i][1]-20),
                            (pos[i][2]+20,pos[i][1]), emotions[l[i]][1], -1, lineType=cv2.LINE_AA)
            
            cv2.putText(image, f'{emotions[l[i]][0]}', (pos[i][0],pos[i][1]-5),
                            0, 0.6, emotions[l[i]][2], 2, lineType=cv2.LINE_AA)
    
    return image

In [10]:
def capture_and_infer():
    # 웹캠 초기화
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    while True:
        # 웹캠으로부터 이미지 캡쳐
        ret, frame = cap.read()
        if not ret:
            print("이미지를 캡쳐할 수 없습니다.")
            break

        # 캡쳐된 이미지에 대한 감정 분석
        processed_image = inference(frame)

        # 결과 이미지 보여주기
        cv2.imshow('Emotion Detection', processed_image)

        # 'q' 키를 누르면 반복문 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 자원 해제
    cap.release()
    cv2.destroyAllWindows()

# 함수 실행
capture_and_infer()

1/1 [==============================] - 0s 413ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 11ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 11ms/step
4
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 15ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 11ms/step
4
1/1 [==============================] - 0s 11ms/step
3
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 11ms/step
4
1/1 [======================